Basic (Shallow) Neural Network Design:

Reid Zaffino (zaffino@uwindsor.ca) 2021-06

Resources from Coursera "Neural Networks and Deep Learning" Course

$$z^{[1] (i)} =  W^{[1]} x^{(i)} + b^{[1]}\tag{1}$$

$$a^{[1] (i)} = \tanh(z^{[1] (i)})\tag{2}$$

$$z^{[2] (i)} = W^{[2]} a^{[1] (i)} + b^{[2]}\tag{3}$$

$$\hat{y}^{(i)} = a^{[2] (i)} = \sigma(z^{ [2] (i)})\tag{4}$$

$$y^{(i)}_{prediction} = \begin{cases} 1 & \mbox{if } a^{[2](i)} > 0.5 \\ 0 & \mbox{otherwise } \end{cases}\tag{5}$$

$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large\left(\small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right)  \large  \right) \small \tag{6}$$

In [1]:
import numpy as np

In [2]:
def sigmoid(x):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """
    
    s = 1/(1 + np.exp(-x))
    
    return s

In [3]:
def layer_sizes(X, Y, n_h):
    """
    Arguments:
    X -- input dataset of shape (input size, number of examples)
    Y -- labels of shape (output size, number of examples)
    
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    
    n_x = X.shape[0]
    n_h = n_h
    n_y = Y.shape[0]
    
    return (n_x, n_h, n_y)

In [4]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

Forward Propagation:

$$Z^{[1]} =  W^{[1]} X + b^{[1]}\tag{1}$$

$$A^{[1]} = \tanh(Z^{[1]})\tag{2}$$

$$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}\tag{3}$$

$$\hat{Y} = A^{[2]} = \sigma(Z^{[2]})\tag{4}$$

In [5]:
def forward_propagation(X, parameters):
    """
    Argument:
    X -- input data of size (n_x, m)
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

Cost Function:
$$J = - \frac{1}{m} \sum\limits_{i = 1}^{m} \large{(} \small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right) \large{)} \small\tag{6}$$

In [6]:
def compute_cost(A2, Y):
    """
    Computes the cross-entropy cost given in equation (13)
    
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (1, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost given equation (13)
    
    """
    
    m = Y.shape[1] # number of examples

    # Compute the cross-entropy cost
    
    logprobs = np.multiply(np.log(A2), Y) + np.multiply(np.log(1 - A2), 1 - Y)
    cost = (-1/m)*np.sum(logprobs)
    
    cost = float(np.squeeze(cost))  # makes sure cost is the dimension we expect. 
                                    # E.g., turns [[17]] into 17 
    
    return cost

Backward Propagation:

$$dZ^{[2]} =  A^{[2]} - Y\tag{7}$$

$$dW^{[2]} = \frac{1}{m} dZ^{[2]}(A^{[1]})^{T}\tag{8}$$

$$db^{[2]} = \frac{1}{m} \sum{dZ^{[2]}}\tag{9}$$

$$dZ^{[1]} = W^{[2]T}dZ^{[2]}*g^{[1]'}(Z^{[1]})\tag{10}$$

$$dW^{[1]} = \frac{1}{m} dZ^{[1]}X^{T}\tag{11}$$

$$db^{[1]} = \frac{1}{m} \sum{dZ^{[1]}}\tag{12}$$

In [7]:
def backward_propagation(parameters, cache, X, Y):
    """
    Implement the backward propagation using the instructions above.
    
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data of shape (2, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = X.shape[1]
    
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    
    A1 = cache["A1"]
    A2 = cache["A2"]
    
    dZ2 = A2 - Y
    dW2 = (1/m)*(np.dot(dZ2, A1.T))
    db2 = (1/m)*np.sum(dZ2, axis = 1, keepdims = True)
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2))
    dW1 = (1/m)*np.dot(dZ1, X.T)
    db1 = (1/m)*np.sum(dZ1, axis = 1, keepdims = True)
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [8]:
def update_parameters(parameters, grads, learning_rate = 1):
    """
    Updates parameters using the gradient descent update rule given above
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    
    W1 = W1 - dW1 * learning_rate
    b1 = b1 - db1 * learning_rate
    W2 = W2 - dW2 * learning_rate
    b2 = b2 - db2 * learning_rate
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [9]:
def nn_model(X, Y, n_h, num_iterations = 10000, print_cost=False):
    """
    Arguments:
    X -- dataset of shape (2, number of examples)
    Y -- labels of shape (1, number of examples)
    n_h -- size of the hidden layer
    num_iterations -- Number of iterations in gradient descent loop
    print_cost -- if True, print the cost every 50 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    np.random.seed(3)
    n_x = layer_sizes(X, Y, n_h)[0]
    n_y = layer_sizes(X, Y, n_h)[2]

    parameters = initialize_parameters(n_x, n_h, n_y)

    for i in range(0, num_iterations):
        
        A2, cache = forward_propagation(X, parameters)
        cost = compute_cost(A2, Y)
        grads = backward_propagation(parameters, cache, X, Y)
        parameters = update_parameters(parameters, grads)
        
        # Print the cost every 50 iterations
        if print_cost and i % 50 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

Prediction:

$$y^{(i)}_{prediction} = \begin{cases} 1 & \mbox{if } a^{[2](i)} > 0.5 \\ 0 & \mbox{otherwise } \end{cases}\tag{5}$$

In [10]:
def predict(parameters, X):
    """
    Using the learned parameters, predicts a class for each example in X
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (n_x, m)
    
    Returns
    predictions -- vector of predictions of our model (0 / 1)
    """
    
    A2, cache = forward_propagation(X, parameters)
    predictions = (A2 > 0.5).astype(int)
    
    return predictions

In [11]:
X = np.array([
    [1, 1, 0, 0],
    [0, 1, 0, 1],
    [0, 0, 1, 1],
    [1, 0, 1, 0],
    [1, 0, 0, 1],
    [0, 1, 1, 0]
]).T

Y = np.array([
    [0, 1],
    [0, 0],
    [0, 1],
    [0, 0],
    [1, 0],
    [1, 1],
]).T

In [12]:
n1 = nn_model(X, Y, 1, num_iterations=200)
n2 = nn_model(X, Y, 2, num_iterations=200)
n3 = nn_model(X, Y, 3, num_iterations=200)
n4 = nn_model(X, Y, 4, num_iterations=200)
n5 = nn_model(X, Y, 5, num_iterations=200)

In [13]:
p1 = predict(n1, X)
p2 = predict(n2, X)
p3 = predict(n3, X)
p4 = predict(n4, X)
p5 = predict(n5, X)

In [14]:
Yre = Y.reshape(1, Y.shape[0] * Y.shape[1])
p1re = p1.reshape(1, p1.shape[0] * p1.shape[1])
p2re = p2.reshape(1, p2.shape[0] * p2.shape[1])
p3re = p3.reshape(1, p3.shape[0] * p3.shape[1])
p4re = p4.reshape(1, p4.shape[0] * p4.shape[1])
p5re = p5.reshape(1, p5.shape[0] * p5.shape[1])

In [15]:
print ('Accuracy of nn1 on training data: %d' % float((np.dot(Yre, p1re.T) + np.dot(1 - Yre, 1 - p1re.T)) / float(Yre.size) * 100) + '%')
print ('Accuracy of nn2 on training data: %d' % float((np.dot(Yre, p2re.T) + np.dot(1 - Yre, 1 - p2re.T)) / float(Yre.size) * 100) + '%')
print ('Accuracy of nn3 on training data: %d' % float((np.dot(Yre, p3re.T) + np.dot(1 - Yre, 1 - p3re.T)) / float(Yre.size) * 100) + '%')
print ('Accuracy of nn4 on training data: %d' % float((np.dot(Yre, p4re.T) + np.dot(1 - Yre, 1 - p4re.T)) / float(Yre.size) * 100) + '%')
print ('Accuracy of nn5 on training data: %d' % float((np.dot(Yre, p5re.T) + np.dot(1 - Yre, 1 - p5re.T)) / float(Yre.size) * 100) + '%')

Accuracy of nn1 on training data: 75%
Accuracy of nn2 on training data: 83%
Accuracy of nn3 on training data: 100%
Accuracy of nn4 on training data: 100%
Accuracy of nn5 on training data: 100%
